In [ ]:
/storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from torch.autograd import Variable

data = np.genfromtxt('/content/Biomarker_Clinical_Data_Images.csv', delimiter=',', skip_header=1, usecols=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20))

avgArr = np.zeros([int(data.shape[0]/49), data.shape[1]])

for i in range(0, int(data.shape[0]/49)):
  start = 49*i+1
  scans = data[start:start+48,:]
  avgArr[i] = scans.mean(axis=0)

np.set_printoptions(precision=3, suppress=True)

presence = np.ceil(avgArr)

biomarkerAvg, temp = np.hsplit(avgArr, np.array([16]))
__, scores = np.hsplit(temp, np.array([1]))
biomarkerPresence, __ = np.hsplit(presence, np.array([16]))

avg_train, avg_test, score_train, score_test = train_test_split(biomarkerAvg, scores, train_size = 150, test_size=42, random_state=0)
presence_train, presence_test, __, __ = train_test_split(biomarkerPresence, scores, train_size = 150, test_size=42, random_state=0)

score_train = (score_train - score_train.min(0))/score_train.ptp(0)
score_test = (score_test - score_test.min(0))/score_test.ptp(0)



In [ ]:
torch_avg_train = Variable(torch.from_numpy(avg_train).float())
torch_avg_test = Variable(torch.from_numpy(avg_test).float())
torch_presence_train = Variable(torch.from_numpy(presence_train).float())
torch_presence_test = Variable(torch.from_numpy(presence_test).float())
torch_score_train = Variable(torch.from_numpy(score_train).float())
torch_score_test = Variable(torch.from_numpy(score_test).float())

class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(2, 4)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(4, 16)
    def forward(self, x):
        X = self.linear(x)
        X2 = self.relu(X)
        y_pred = self.linear2(X2)
        return y_pred

In [ ]:
del model
model = LinearRegressionModel()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
for epoch in range(5000):

    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y = model(torch_score_train)

    # Compute and print loss
    loss = loss_fn(pred_y, torch_presence_train)

    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch % 20 == 0):
      print('epoch {}, loss {}'.format(epoch, loss.item()))

test_pred = model(torch_score_test)
np.set_printoptions(precision=3, suppress=True)
torch.set_printoptions(threshold=10_000)
print(test_pred)

epoch 0, loss 0.11827762424945831
epoch 20, loss 0.11827044188976288
epoch 40, loss 0.11826324462890625
epoch 60, loss 0.11825606971979141
epoch 80, loss 0.11824891716241837
epoch 100, loss 0.11824175715446472
epoch 120, loss 0.11823461204767227
epoch 140, loss 0.11822747439146042
epoch 160, loss 0.11822034418582916
epoch 180, loss 0.1182132363319397
epoch 200, loss 0.11820604652166367
epoch 220, loss 0.11819882690906525
epoch 240, loss 0.11819161474704742
epoch 260, loss 0.1181844174861908
epoch 280, loss 0.11817722022533417
epoch 300, loss 0.11817005276679993
epoch 320, loss 0.11816287785768509
epoch 340, loss 0.11815571784973145
epoch 360, loss 0.11814858764410019
epoch 380, loss 0.11814142763614655
epoch 400, loss 0.11813430488109589
epoch 420, loss 0.11812718957662582
epoch 440, loss 0.11812008172273636
epoch 460, loss 0.11811298131942749
epoch 480, loss 0.11810588836669922
epoch 500, loss 0.11809880286455154
epoch 520, loss 0.11809173226356506
epoch 540, loss 0.11808466911315918
